# Introdução ao SciPy

## Funções especiais

A biblioteca **SciPy** define diversas funções especiais (). O exemplo abaixo constroi o gráfico da 1ª função Bessel.

In [ ]:
from scipy import special
import numpy as np
import matplotlib.pyplot as plt

def drumhead_height(n, k, distance, angle, t):
   kth_zero = special.jn_zeros(n, k)[-1]   # zeros na j-esima funçao Bessel
   return np.cos(t) * np.cos(n*angle) * special.jn(n, distance*kth_zero)

theta = np.r_[0:2*np.pi:50j]
radius = np.r_[0:1:50j]
x = np.array([r * np.cos(theta) for r in radius])
y = np.array([r * np.sin(theta) for r in radius])
z = np.array([drumhead_height(1, 1, r, theta, 0.5) for r in radius])

fig = plt.figure()
ax = fig.add_axes(rect=(0, 0.05, 0.95, 0.95), projection='3d')
ax.plot_surface(x, y, z, rstride=1, cstride=1, cmap='RdBu_r', vmin=-0.5, vmax=0.5)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_xticks(np.arange(-1, 1.1, 0.5))
ax.set_yticks(np.arange(-1, 1.1, 0.5))
ax.set_zlabel('Z')
plt.show()

## Integração

A integração de funções também é bastante simples utilizando a biblioteca. Por exemplo, o código abaixo calcula a integral

$$
\int_0^1 a x^2 + b \quad \mathrm{d} x
$$

para $a$ e $b$ passados por parâmetro. O resultado é uma tupla em que o 1º valor é o resultado da integral e o 2º valor é o erro.

In [ ]:
# É possível calcular integrais...
from scipy.integrate import quad
def integrand(x, a, b):
    return a*x**2 + b

a = 2
b = 1
I = quad(integrand, 0, 1, args=(a,b))
I

# O resultado é uma tupla. O primeiro valor é o resultado e o segundo o erro.

Para calcular integrais com limites infinitos, basta usar `np.inf`. Por exemplo, a integral

$$
\int_1^\infty \frac{e^{-x t}}{t^n} \quad \mathrm{d} t
$$

pode ser calculada como

In [ ]:
from scipy.integrate import quad
import numpy as np
def integrand(t, n, x):
    return np.exp(-x*t) / t**n

def expint(n, x):
    return quad(integrand, 1, np.inf, args=(n, x))[0]

vec_expint = np.vectorize(expint)

vec_expint(3, np.arange(1.0, 4.0, 0.5))

Esta integral é uma função especial chamada **integral exponencial** e pode ser calculada diretamente:

In [ ]:
import scipy.special as special
special.expn(3, np.arange(1.0, 4.0, 0.5))

Uma **função lambda** é uma construção implementada nas linguagens de programação modernas para implementar o conceito de *cálculo lambda*. Em resumo, funções podem ser definidas no formado lambda da seguinte forma:

In [ ]:
# A função f(x)...
def f(x):
  return 2 * x

# ... pode ser definida como

f = lambda x: 2 * x

# Da mesma forma, a função
def g(x, y):
  return x * y

# pode ser definida como

g = lambda x, y: x * y

# A vantagem é poder definir funções anônimas.

As funções lambda podem ser usadas para se calcular integrais múltiplas usando `scipy.dblquad` (integral dupla), `scipy.tplquad` (integral tripla) ou `scipy.nquad` (integral de ordem $n$).

Por exemplo, para calcular a integral

$$
\int_0^\infty \int_1^\infty \frac{e^{-x t}}{t^n} \quad \mathrm{d}t \ \mathrm{d} x = \frac{1}{n}
$$

é possível fazer

In [ ]:
from scipy.integrate import nquad

N = 5

nquad(lambda x, t: np.exp(-x*t)/(t**N), [[0, np.inf], [1, np.inf]])

De forma análoga, a integral

$$
\int_0^{1/2} \int_0^{1 - 2 y} x y \quad \mathrm{d}x \ \mathrm{d} y = \frac{1}{96} = 0.0104
$$

pode ser calculada como

In [ ]:
from scipy.integrate import dblquad
area = dblquad(lambda x, y: x*y, 0, 0.5, lambda y: 0, lambda y: 1 - 2*y)
area

## Otimização

A biblioteca **numpy** permite a otimização de funções (análogo ao compando `optim()` do R. Por exemplo, para minimizar a *função de Rosembrock*

$$
f(\boldsymbol{x}, a, b) = \sum_{i = 1}^{N - 1} a (x_{i + 1} - x_i^2)^2 + (1 - x_i)^2 + b
$$

para $a = 0.5$ e $b = 1$ é possível fazer:

In [ ]:
from scipy.optimize import minimize

def rosen_with_args(x, a, b):
    """The Rosenbrock function with additional arguments"""
    return sum(a*(x[1:]-x[:-1]**2.0)**2.0 + (1-x[:-1])**2.0) + b

x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])
res = minimize(rosen_with_args, x0, method='nelder-mead',
               args=(0.5, 1.), options={'xatol': 1e-8, 'disp': True})

In [ ]:
print(res.x)

Diferentes métodos de otimização podem ser consultados na documentação da biblioteca. Inclusive métodos para otimização multidimensional e com restrições.

## Interpolação

A biblioteca **scipy** também permite realizar interpolação de funções com base em alguns pontos de dados:

In [ ]:
import numpy as np
x = np.linspace(0, 10, num=11)
y = np.cos(-x**2 / 9.0)

xnew = np.linspace(0, 10, num=1001)
ynew = np.interp(xnew, x, y)

import matplotlib.pyplot as plt
plt.plot(xnew, ynew, '-', label='linear interp')
plt.plot(x, y, 'o', label='data')
plt.legend(loc='best')
plt.show()

In [ ]:
from scipy.interpolate import CubicSpline
x = np.linspace(0, 10, num=11)
y = np.cos(-x**2 / 9.)
spl = CubicSpline(x, y)

import matplotlib.pyplot as plt
fig, ax = plt.subplots(4, 1, figsize=(5, 7))
xnew = np.linspace(0, 10, num=1001)
ax[0].plot(xnew, spl(xnew))
ax[0].plot(x, y, 'o', label='data')
ax[1].plot(xnew, spl(xnew, nu=1), '--', label='1st derivative')
ax[2].plot(xnew, spl(xnew, nu=2), '--', label='2nd derivative')
ax[3].plot(xnew, spl(xnew, nu=3), '--', label='3rd derivative')
for j in range(4):
    ax[j].legend(loc='best')
plt.tight_layout()
plt.show()

Também é possível interpolar dados bidimensionais (pontos em uma malha).

In [ ]:
import numpy as np
from scipy.interpolate import RBFInterpolator
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import matplotlib.pyplot as plt

# Interpolar uma função 2D
def func(x, y):
    return x*(1-x)*np.cos(4*np.pi*x) * np.sin(4*np.pi*y**2)**2

# em uma malha [0, 1] x [0, 1]
grid_x, grid_y = np.meshgrid(np.linspace(0, 1, 100),
                             np.linspace(0, 1, 200), indexing='ij')

# conhecendo somente 1.000 valores na malha (de tamanho 20.000)
rng = np.random.default_rng()
points = rng.random((1000, 2))
values = func(points[:,0], points[:,1])

# A interpolação é feita usando griddata
grid_z0 = griddata(points, values, (grid_x, grid_y), method='nearest')
grid_z1 = griddata(points, values, (grid_x, grid_y), method='linear')
grid_z2 = griddata(points, values, (grid_x, grid_y), method='cubic')

# O resultado...

plt.subplot(221)
plt.imshow(func(grid_x, grid_y).T, extent=(0, 1, 0, 1), origin='lower')
plt.plot(points[:, 0], points[:, 1], 'k.', ms=1)   # data
plt.title('Original')
plt.subplot(222)
plt.imshow(grid_z0.T, extent=(0, 1, 0, 1), origin='lower')
plt.title('Ponto mais próximo')
plt.subplot(223)
plt.imshow(grid_z1.T, extent=(0, 1, 0, 1), origin='lower')
plt.title('Linear')
plt.subplot(224)
plt.imshow(grid_z2.T, extent=(0, 1, 0, 1), origin='lower')
plt.title('Cúbico')
plt.gcf().set_size_inches(6, 6)
plt.show()